In [1]:
import numpy as np
import h5py
import math
from sklearn.decomposition import PCA
from pca import pca
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import peakutils
import os, sys
from BaselineRemoval import BaselineRemoval
from sklearn.cluster import DBSCAN
import scipy
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [2]:
class HiddenPrints:
	def __enter__(self):
		self._original_stdout = sys.stdout
		sys.stdout = open(os.devnull, 'w')

In [3]:
def __exit__(self, exc_type, exc_val, exc_tb):
	sys.stdout.close()
	sys.stdout = self._original_stdout

In [4]:
def remove_outliers(xset, yset, zset, eset, threshold):
	"""
	Uses DBSCAN to find and remove outliers in 3D data
	"""
	data = np.array([xset.T, yset.T, zset.T]).T
	DBSCAN_cluster = DBSCAN(eps=7, min_samples=10).fit(data)
	out_of_cluster_index = np.where(DBSCAN_cluster.labels_==-1)
	del data
	rev = out_of_cluster_index[0][::-1]
	#if len(out_of_cluster_index[0]) > 0:
	for i in rev:
		xset = np.delete(xset, i)
		yset = np.delete(yset, i)
		zset = np.delete(zset, i)
		eset = np.delete(eset, i)
	#if len(xset) <= threshold:
	#	veto = True
	#else:
    #veto = False
	return xset, yset, zset, eset, False

In [5]:
print('Veto on Length @ 70 mm')
print('Veto on Integrated Charge @ 800k')
def track_len(xset, yset, zset):
    """
    Uses PCA to find the length of a track
    """
    veto_on_length = False
 
    # Form data matrix
    data = np.concatenate((xset[:, np.newaxis], 
                           yset[:, np.newaxis], 
                           zset[:, np.newaxis]), 
                           axis=1)

    # Use PCA to find track length
    pca = PCA(n_components=3)
    principalComponents = pca.fit(data)
    principalComponents = pca.transform(data)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
    
    track_len = 2.35*principalDf.std()[0]
    track_width = 2.35*principalDf.std()[1]
    track_depth = 2.35*principalDf.std()[2]
    #if track_len > 70:
    #    veto_on_length = True
    
    return track_len, veto_on_length, track_width, track_depth

Veto on Length @ 70 mm
Veto on Integrated Charge @ 800k


In [6]:
def main(h5file, threshold):
    """
    This functions does the following: 
    - Converts h5 files into ndarrays. 
    - Removes outliers.
    - Calls PCA to return track length.
    - Sums mesh signal to return energy.
    """
    # Converts h5 files into ndarrays, and output each event dataset as a separte list
    num_events = int(len(list(h5file.keys()))/2) 
    
    len_list = []
    width_list = []
    depth_list = []
    good_events = []
    tot_energy = []
    tracemax_list = []
    tracedev_list = []
    pads_list = []
    
    skipped_events = 0
    veto_events = 0
    
    pbar = tqdm(total=num_events+1)
    for i in range(0, num_events):
        str_event = f"Event_[{i}]"
        
        # Apply pad threshold
        event = np.array(h5file[str_event][:])
        if len(event) <= threshold:
            skipped_events += 1
            pbar.update(n=1)
            continue
            
        # Make copy of datasets
        dset_0_copyx = event['x']
        dset_0_copyy = event['y'] 
        dset_0_copyz = event['z'] - min(event['z'])
        dset_0_copye = event['A']

        
        # Apply veto condition
        R = 36                           # Radius of the pad plane
        r = np.sqrt(dset_0_copyx**2 + dset_0_copyy**2)
        statements = np.greater(r, R)    # Check if any point lies outside of R
      
        if np.any(statements) == True:
            veto_events += 1
            pbar.update(n=1)
            continue
        
        
        # Call remove_outliers to get dataset w/ outliers removed
        dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, veto = remove_outliers(dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, threshold)
        if veto == True:
            skipped_events += 1
            pbar.update(n=1)
            continue

        
        # Call track_len() to create lists of all track lengths
        length, veto_on_length, width, depth = track_len(dset_0_copyx, dset_0_copyy, dset_0_copyz)
        if veto_on_length == True:
            veto_events += 1
            pbar.update(n=1)
            continue

        
       	str_trace = f"Trace_[{i}]"
        trace = np.array(h5file[str_trace][:])
        max_val = np.argmax(trace)
        low_bound = max_val - 75
        if low_bound < 0:
            low_bound = 5
        upper_bound = max_val + 75
        if upper_bound > 511:
            upper_bound = 506
        trace = trace[low_bound:upper_bound]

        polynomial_degree=2 
        baseObj=BaselineRemoval(trace)
        trace=baseObj.IModPoly(polynomial_degree)

        #if np.sum(trace) > 800000:
        #    veto_events += 1
        #    pbar.update(n=1)
        #    continue

        len_list.append(length)
        width_list.append(width)
        depth_list.append(depth)
        tot_energy.append(np.sum(trace))
        tracemax_list.append(np.max(trace))
        tracedev_list.append(np.std(trace))
        pads_list.append(len(trace[trace > 0]))

        # Track event number of good events
        good_events.append(i)  
        pbar.update(n=1)

    return (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list)

In [7]:
# assign directory
directory = "C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/"

output_df = pd.DataFrame(columns=['file','event','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

file_names = []
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if f[-3:] == '.h5':
            print(f)
            file_names.append(filename)
            h5f = h5py.File(directory+filename, 'r')
            (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list) = main(h5file=h5f, threshold=15)
            for event in range(len(tot_energy)):
                output_df = output_df.append({'file' : filename, 
                                              'event' : event, 
                                              'length' : len_list[event], 
                                              'width' : width_list[event], 
                                              'depth' : depth_list[event], 
                                              'tracesum' : tot_energy[event], 
                                              'tracemax' : tracemax_list[event], 
                                              'tracedev' : tracedev_list[event], 
                                              'padnum' : pads_list[event]}
                                             ,ignore_index = True)


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/test_1000AA.h5


 95%|█████████▌| 20/21 [00:01<00:00, 12.97it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/test_1200proton500alpha.h5


 95%|█████████▌| 20/21 [00:01<00:00, 12.16it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/test_500alpha.h5


 95%|█████████▌| 20/21 [00:01<00:00, 16.09it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/test_800PP.h5


 95%|█████████▌| 20/21 [00:01<00:00, 11.80it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/test_800proton.h5


 95%|█████████▌| 20/21 [00:01<00:00, 17.39it/s]


In [8]:
for file in file_names:
    print(file)
    print('length:\t', output_df[output_df['file'] == file]['length'].mean(),'\t', output_df[output_df['file'] == file]['length'].std())
    print('width:\t', output_df[output_df['file'] == file]['width'].mean(),'\t', output_df[output_df['file'] == file]['width'].std())
    print('depth:\t', output_df[output_df['file'] == file]['depth'].mean(),'\t', output_df[output_df['file'] == file]['depth'].std())
    print('tracesum:\t', output_df[output_df['file'] == file]['tracesum'].mean(),'\t', output_df[output_df['file'] == file]['tracesum'].std())
    print('tracemax:\t', output_df[output_df['file'] == file]['tracemax'].mean(),'\t', output_df[output_df['file'] == file]['tracemax'].std())
    print('tracedev:\t', output_df[output_df['file'] == file]['tracedev'].mean(),'\t', output_df[output_df['file'] == file]['tracedev'].std())
    print('padnum:\t', output_df[output_df['file'] == file]['padnum'].mean(),'\t', output_df[output_df['file'] == file]['padnum'].std())
    print()

test_1000AA.h5
length:	 12.854182468417406 	 1.234292535250228
width:	 11.32933332281868 	 1.1733539683701781
depth:	 3.489283242051298 	 0.6246687687152638
tracesum:	 1136218.7227491443 	 4941.553338063969
tracemax:	 94085.81097318664 	 7250.701672981606
tracedev:	 21225.829628087704 	 941.0217809746528
padnum:	 110.65 	 2.4553914898486258

test_1200proton500alpha.h5
length:	 29.33395548862193 	 0.9531753486521912
width:	 11.04855454553013 	 1.329693434266354
depth:	 4.207750856062226 	 0.9331898564105945
tracesum:	 1282507.7930782863 	 6670.947741358467
tracemax:	 87417.65821503835 	 17336.78657707007
tracedev:	 21580.062263199015 	 2228.630223413188
padnum:	 98.76470588235294 	 10.951765906491438

test_500alpha.h5
length:	 10.937878768787058 	 1.024929658365177
width:	 10.281309210032825 	 1.0069768707900644
depth:	 3.9772690334546175 	 0.9173206089857148
tracesum:	 415395.7403382207 	 2355.097586658481
tracemax:	 34045.76139989777 	 2679.074130350298
tracedev:	 7717.5834161334105 	